In [6]:
from pathlib import Path
import pandas as pd
from pytorch_forecasting.data.timeseries import TimeSeriesDataSet
import os

In [2]:
zone='SUD'
batch_size=256
test_batch_size=1000

num_workers = 0
print(f'Building {zone} data loader with {num_workers} workers')

Building SUD data loader with 0 workers


In [7]:
PATH = 'data'
FILE = os.path.join(PATH, f'wind_{zone}.csv')

data = pd.read_csv(FILE)

In [8]:
min_encoder_length = 0
max_encoder_length = 400
min_prediction_length = 1
max_prediction_length = 7
training_cutoff = data["time_idx"].max() - max_prediction_length

In [9]:
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx='time_idx',
    target='energy',
    group_ids=['energy'],
    min_encoder_length=min_encoder_length,
    max_encoder_length=max_encoder_length,
    min_prediction_length=min_prediction_length,
    max_prediction_length=max_prediction_length,
    # target_normalizer=,
    allow_missing_timesteps=True,
    time_varying_unknown_reals=['energy']
)

testing = TimeSeriesDataSet.from_dataset(
    training, data, predict=True, stop_randomization=True
)

/Users/nicolafraccarolo/env/wind/lib/python3.10/site-packages/pytorch_forecasting/data/timeseries.py:1282: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 196 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__energy': 0.2}, {'__group_id__energy': 0.3}, {'__group_id__energy': 0.5}, {'__group_id__energy': 1.6}, {'__group_id__energy': 1.9}, {'__group_id__energy': 2.1}, {'__group_id__energy': 4.2}, {'__group_id__energy': 4.4}, {'__group_id__energy': 5.6}, {'__group_id__energy': 5.7}]
  warnings.warn(


In [34]:
# create dataloaders for model
train_loader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=0
)
test_loader = testing.to_dataloader(
    train=False, batch_size=test_batch_size, num_workers=0
)

In [35]:
len(train_loader)

16

In [63]:
type(next(iter(train_loader))[0]), len(next(iter(train_loader))[0])

(dict, 11)

In [64]:
type(next(iter(train_loader))[1]), len(next(iter(train_loader))[1])

(tuple, 2)

In [65]:
next(iter(train_loader))[1]

(tensor([[ 2.0000,  2.0000,  2.0000,  ...,  2.0000,  2.0000,  2.0000],
         [20.2000, 20.2000, 20.2000,  ..., 20.2000, 20.2000, 20.2000],
         [ 6.9000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [26.7000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [26.7000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.5000,  1.5000,  1.5000,  ...,  1.5000,  1.5000,  1.5000]]),
 None)

In [52]:
next(iter(train_loader))[0].keys()

dict_keys(['encoder_cat', 'encoder_cont', 'encoder_target', 'encoder_lengths', 'decoder_cat', 'decoder_cont', 'decoder_target', 'decoder_lengths', 'decoder_time_idx', 'groups', 'target_scale'])

In [60]:
for k in next(iter(train_loader))[0].keys():
    print(k, '\t', next(iter(train_loader))[0][k].shape)

encoder_cat 	 torch.Size([256, 400, 0])
encoder_cont 	 torch.Size([256, 400, 2])
encoder_target 	 torch.Size([256, 399])
encoder_lengths 	 torch.Size([256])
decoder_cat 	 torch.Size([256, 7, 0])
decoder_cont 	 torch.Size([256, 7, 2])
decoder_target 	 torch.Size([256, 7])
decoder_lengths 	 torch.Size([256])
decoder_time_idx 	 torch.Size([256, 7])
groups 	 torch.Size([256, 1])
target_scale 	 torch.Size([256, 2])
